In [ ]:
import os
import json
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from collections import defaultdict
from urllib.request import Request, urlopen
import pandas as pd

load_path = '/data/zhaohan/adv-reasoning/data/cyberkg-raw/cve_ex_links/top1.www.securityfocus.com.60510.csv'
save_path = '/data/zhaohan/adv-reasoning/data/cyberkg-raw/cve_patch/www.securityfocus.com'

saved_info = defaultdict(list)

data = pd.read_csv(load_path, delimiter='|',  index_col=0, header=0)
counter = 0
for index, row in tqdm(data.iterrows(), desc='reading links from www.securityfocus.com'):
    cve_ids = row['cve-ids']
    ex_url = row['ex_url']
    types = row['types']
    for key in row.keys():
        saved_info[key].append(row[key])
    
    solution_text = ''
    if ex_url.startswith(('http://www.securityfocus.com/bid/', 
                          'https://www.securityfocus.com/bid/')):
        # those are formal web records at least with 'solution' section
        solution_url = os.path.join(ex_url, 'solution')
        try:
            req = Request(solution_url, headers={'User-Agent': 'Mozilla/5.0'})
            webpage = urlopen(req).read()
            html = BeautifulSoup(webpage, 'html.parser')
            for div in html.find_all('div', attrs={'id': 'vulnerability'}):
                solution_text = div.text.split('Solution:')[-1].strip()
                solution_text = solution_text.replace("\n", "<LINE BREAK>")
                solution_text = solution_text.replace("|", "<OR SPLITTER>")
        except:
            print(ex_url)
    
    elif ex_url.startswith(('http://www.securityfocus.com/archive/', 
                            'https://www.securityfocus.com/archive/',
                            'http://www.securityfocus.com//archive/')):
        # those are textual posts that may contain solution or not. 
        pass
    elif ex_url.startswith(('http://www.securityfocus.com/advisories/')):
        # those are 'malformed request' webpage, looks useless
        pass
    else:
        # those are useless or invalid links
        pass
            
    saved_info['solution'].append(solution_text)
    
    if len(saved_info['cve-ids']) % 500 == 0:
        df = pd.DataFrame.from_dict(saved_info)
        os.makedirs(save_path, exist_ok=True)
        df.to_csv(os.path.join(save_path, '%d.csv' % counter), sep='|')
        
        saved_info = defaultdict(list)
        counter += 1



reading links from www.securityfocus.com: 91it [03:21, 10.19s/it]

http://www.securityfocus.com/bid/18655


reading links from www.securityfocus.com: 341it [11:32,  1.24it/s]